In [1]:
import pandas as pd
from datetime import date, datetime
import httpx

### Variáveis do projeto

In [2]:
arquivo = "./arquivos/cadastro_de_empregadores-atualizacao-extraord-09-mar-2023.xlsx"

In [3]:
columns = [
    "id",
    "ano_acao_fiscal",
    "uf",
    "empregador",
    "cpf_cpnj",
    "estabelecimento",
    "trabalhadores_envolvidos",
    "cnae",
    "data_decisao_administrativa",
    "data_inclusao_cadastro",
]

# Funções auxiliares

In [4]:
def tratamento_de_datas(data):
    if isinstance(data, datetime):
        return data.date()
    _data = data.replace(".","").split(" ")

    for d in reversed(_data):
        try:
            d = d.split("/")
            return date(day=int(d[0]), month=int(d[1]), year=int(d[2]))
        except Exception:
            ...

### Tratamento do DataFrame

In [5]:
# Ler o arquivo
# Dica para não converter colunas automaticamente: https://stackoverflow.com/questions/59258431/how-can-i-stop-pandas-from-automatically-converting-my-date-format
# Dica para nomear colunas: https://stackoverflow.com/questions/17335649/how-to-specify-column-names-while-reading-an-excel-file-using-pandas
df = pd.read_excel(arquivo)

In [6]:
# Altera o nome das colunas
df.columns = columns

In [7]:
# Procura e preenche a data_do_arquivo
for l in df.id:
    try:
        data_do_arquivo = tratamento_de_datas(l)
    except Exception:
        ...
    else:
        break

In [8]:
data_do_arquivo

datetime.date(2023, 3, 9)

In [9]:
# Elimina os registros vazios
# TODO: Melhorar a exclusão da lista
df.dropna(inplace=True)

In [10]:
# Resetar os índices
df.reset_index(drop=True, inplace=True)

In [11]:
# Elimina o cabeçalho do arquivo
df.drop(0, inplace=True)

In [12]:
# Tratamento de datas
df.data_decisao_administrativa = df.data_decisao_administrativa.apply(tratamento_de_datas)

# TODO: Não sobrescrever a coluna data_inclusao_cadastro para não perder as demais datas
df.data_inclusao_cadastro = df.data_inclusao_cadastro.apply(tratamento_de_datas)

In [13]:
# Cria a data do arquivo
df["data_arquivo"] = data_do_arquivo

In [14]:
# Cria a data da carga
df["data_carga"] = date.today()

# Próximos passos

In [16]:
# Faz o request na página -> https://www.gov.br/trabalho-e-previdencia/pt-br/composicao/orgaos-especificos/secretaria-de-trabalho/inspecao/areas-de-atuacao/combate-ao-trabalho-escravo-e-analogo-ao-de-escravo
# Encontro o arquivo XLSX
# Baixo o arquivo XLSX
# Comparo o arquivo baixado com o único arquivo na pasta arquivos. Se o arquivo for igual (HASH), interrompe. Se o arquivo for diferente, continua o processamento
# Move o arquivo que já estava na pasta "arquivos" para "processados"
# Salva o novo arquivo na pasta arquivos
# Schedular a execução (Rocketry)
# Logging


# TODO: Criar função para capturar e comparar o arquivo
arquivo = httpx.get('https://www.gov.br/trabalho-e-previdencia/pt-br/composicao/orgaos-especificos/secretaria-de-trabalho/inspecao/areas-de-atuacao/cadastro_de_empregadores-atualizacao-extraord-09-mar-2023.xlsx')
with open("./arquivos/cadastro_de_empregadores-atualizacao-extraord-09-mar-2023-baixado.xlsx", "wb") as f:
    f.write(arquivo.content)

In [17]:
df

,id,ano_acao_fiscal,uf,empregador,cpf_cpnj,estabelecimento,trabalhadores_envolvidos,cnae,data_decisao_administrativa,data_inclusao_cadastro,data_arquivo,data_carga
1,1,2020,PI,A. de C. Borges,27.090.456/0002-27,"Cerâmica J.A, Rodovia PI, sentido de Barras a ...",9,2341-9/00,2021-12-09,2022-10-05,2023-03-09,2023-04-01
2,2,2018,RN,Ademar Pelonha de Menezes Filho,057.293.644-31,"Sítio Bela Vista I, s/n, zona rural, entre o d...",5,0220-9/99,2019-11-28,2022-10-05,2023-03-09,2023-04-01
3,3,2018,BA,Adilson Bona Vieira,088.062.627-50,Fazenda Dois Rios e Dois Rios I - Rodovia BA-2...,39,0134-2/00,2020-02-27,2021-04-05,2023-03-09,2023-04-01
4,4,2019,MT,Adilson Capanema de Freitas,389.732.001-00,"Fazenda Sonho do Mineiro, Gleba Cruzeiro do Su...",2,0151-2/01,2021-12-13,2022-10-05,2023-03-09,2023-04-01
5,5,2019,SP,Agda Dias da Silva,521.267.221-04,Alojamento e pontos de prostituição de profiss...,12,9700-5/00,2022-05-23,2022-10-05,2023-03-09,2023-04-01
...,...,...,...,...,...,...,...,...,...,...,...,...
171,171,2019,MG,Wagner Antunes Sposito,512.585.606-15,"Fazenda Tamboril-Jacaré, zona rural, Ninheira/MG",23,0210-1/08,2020-02-07,2021-12-23,2023-03-09,2023-04-01
172,172,2020,MG,Wagner Arthur Gonçalves dos Santos,062.012.016-96,"Fazenda Mesas, zona rural, Campos Altos/MG",34,0134-2/00,2021-12-01,2022-04-05,2023-03-09,2023-04-01
173,173,2019,RN,Waldir Luduvino Costa,495.920.844-91,"Carnaubal, zona rural, Carnaubais/RN",4,2209-9/99,2021-09-13,2022-04-05,2023-03-09,2023-04-01
174,174,2019,RS,Wilson Luiz Vanni,056.471.500-04,"Propriedade Rural, Linha Julieta S/N, zona rur...",1,1334-9/99,2021-04-12,2021-10-05,2023-03-09,2023-04-01
